In [1]:
#CLI: aws s3 cp s3://datarozhlas/flightradar-leto19 . --recursive
import json, time, os

In [11]:
'''
["identification"]["row"] :: unique flight id / key
["status"]["live"] :: save only if False
["status"]["generic"]["status"]["text"] :: save only if departed/landed

save:
["aircraft"]["model"] :: aircraft id & name
["flight"]["airline"]["name"] :: airline
departures: ["time"]["scheduled"]["departure"] & ["time"]["real"]["departure"]
arrivals: ["time"]["scheduled"]["arrival"] & ["time"]["real"]["arrival"]
'''
def processPage(page, type): #type: arrival/departure
    pageData = {}
    for entry in page:
        flight = entry["flight"]
        if (flight["status"]["generic"]["status"]["text"] == "departed" or flight["status"]["generic"]["status"]["text"] == "landed") and flight["status"]["live"] == False:
            try: pageData[flight["identification"]["row"]] = {
                "model": flight["aircraft"]["model"],
                "airline": flight["airline"]["name"],
                "delay": flight["time"]["real"][type] - flight["time"]["scheduled"][type]
            }
            except: pass
    return pageData

In [3]:
merged_data = {}

In [12]:
for filename in list(os.walk("."))[0][2]:
    if filename.endswith("json") and "merged" not in filename:
        with open(filename) as file:
            jsonfile = json.load(file)
            for airport in jsonfile:
                if not airport in merged_data: merged_data[airport] = {"arrivals": {}, "departures": {}}
                for page in jsonfile[airport]["departures"]:
                    merged_data[airport]["departures"].update(processPage(jsonfile[airport]["departures"][page], "departure"))
                for page in jsonfile[airport]["arrivals"]:
                    merged_data[airport]["arrivals"].update(processPage(jsonfile[airport]["arrivals"][page], "arrival"))

In [13]:
len(merged_data["prg"]["departures"])

5707

In [14]:
with open("merged_data.json","w",encoding="utf8") as file:
    json.dump(merged_data, file)